In [18]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('../../smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.experiment.databaseObj import Database
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.base import runtime
from smodels.experiment.expSMS import ExpSMS
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V


3.0.0-beta


In [19]:
BSMlist = getParticlesFromSLHA('TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)


[gha, gha~, ghz, ghz~, ghwp, ghwp~, ghwm, ghwm~, ghg, ghg~, xr, xc, xc~, xd, xd~, y1]


In [20]:
slhaFolder = 'TRV1jj_slha_broad'
resultsFolder = 'TRV1jj_results_broad'
File = 'parameters.ini'
!chmod -R +rwx {slhaFolder}


In [21]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}


INFO in databaseObj.loadTextDatabase() in 479: Parsing text database at /home/yoxara/smodels-database/
INFO in databaseObj.createBinaryFile() in 630: /home/yoxara/smodels-database/db3.pcl created.
INFO in modelTester.loadDatabaseResults() in 497: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 645 files with a single process. Messages will be redirected to smodels.log


In [22]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)


In [23]:
print(len(data))


645


In [24]:
#Convert data to flat DataFrame:
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)
smodelsDF = json_normalize(data)
smodelsDF


,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_449_mzp_4700_gvq_0.63.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.715551e+00,0.000000e+00,1.013700e+01,0.000000e+00,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha_broad/run_449_mzp_4700_gvq_0.63.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,7.421450e+00,1.458480e+01,NaN,[TRV1jj],"[(y1, 4700.0)]",CMS-EXO-19-012,NaN,1.300000e+01,1.370000e+02,upperLimit,5.088482e-01,NaN,"[(y1, 1026.4)]"
1,run_333_mzp_4000_gvq_0.19.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.816803e+00,0.000000e+00,4.228500e+00,0.000000e+00,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha_broad/run_333_mzp_4000_gvq_0.19.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,1.411697e+00,1.838170e+01,NaN,[TRV1jj],"[(y1, 4000.0)]",CMS-EXO-19-012,NaN,1.300000e+01,1.370000e+02,upperLimit,7.679907e-02,NaN,"[(y1, 177.02)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,run_454_mzp_4800_gvq_0.22.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",6.069257e-01,0.000000e+00,9.964600e-01,0.000000e+00,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha_broad/run_454_mzp_4800_gvq_0.22.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,3.895343e-01,5.945770e+00,NaN,[TRV1jj],"[(y1, 4800.0)]",CMS-EXO-19-012,NaN,1.300000e+01,1.370000e+02,upperLimit,6.551453e-02,NaN,"[(y1, 239.83)]"
644,run_346_mzp_4100_gvq_0.13.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",1.296949e+00,0.000000e+00,1.623500e+00,0.000000e+00,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha_broad/run_346_mzp_4100_gvq_0.13.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,3.265514e-01,1.338840e+01,NaN,[TRV1jj],"[(y1, 4100.0)]",CMS-EXO-19-012,NaN,1.300000e+01,1.370000e+02,upperLimit,2.439062e-02,NaN,"[(y1, 143.36)]"


In [25]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >=55])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 55])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)


In [26]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)


In [27]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')


In [28]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names


Final number of data points: 645


In [29]:
#Save DataFrame to pickle file:
dataDF.to_csv('TRV1jj_broad.csv')
print(len(dataDF))
dataDF


645


,filename,mass.55,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.55,width.5000001,width.9000002,width.9000003,width.9000004,extpar.1,extpar.2,extpar.3,extpar.4,extpar.5,extpar.6,extpar.7,extpar.8,extpar.9,extpar.10,extpar.11,extpar.12,extpar.13,extpar.14,extpar.15,extpar.16,xsec13TeV(fb).55,"BRs.y1.b,b","BRs.y1.c,c","BRs.y1.q,q","BRs.y1.t-,t+","BRs.y1.xd~,xd",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_449_mzp_4700_gvq_0.63.slha,4.700000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,1.026409e+03,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.013700e+01,1.464230e-01,1.464230e-01,1.464230e-01,1.464214e-01,1.214637e-01,1.013700e+01,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.715551e+00,0.000000e+00,1.013700e+01,0.000000e+00,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha_broad/run_449_mzp_4700_gvq_0.63.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,7.421450e+00,1.458480e+01,NaN,[TRV1jj],"[(y1, 4700.0)]",CMS-EXO-19-012,NaN,1.300000e+01,1.370000e+02,upperLimit,5.088482e-01,NaN,"[(y1, 1026.4)]"
1,run_333_mzp_4000_gvq_0.19.slha,4.000000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,1.770224e+02,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.927000e-01,1.927000e-01,1.927000e-01,1.927000e-01,1.927000e-01,1.927000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.228500e+00,6.677060e-02,6.677060e-02,6.677060e-02,6.676923e-02,5.993778e-01,4.228500e+00,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.816803e+00,0.000000e+00,4.228500e+00,0.000000e+00,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha_broad/run_333_mzp_4000_gvq_0.19.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,1.411697e+00,1.838170e+01,NaN,[TRV1jj],"[(y1, 4000.0)]",CMS-EXO-19-012,NaN,1.300000e+01,1.370000e+02,upperLimit,7.679907e-02,NaN,"[(y1, 177.02)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,run_454_mzp_4800_gvq_0.22.slha,4.800000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,2.398271e+

In [30]:
dataDF['xsec13TeV(fb).55'] *(3 *dataDF['BRs.y1.q,q'] + dataDF['BRs.y1.c,c'] + dataDF['BRs.y1.b,b'] )


0     7.421450e+00
1     1.411697e+00
          ...     
643   3.895343e-01
644   3.265514e-01
Length: 645, dtype: float64

In [31]:
dataDF['ExptRes.result0.theory prediction (fb)']


0     7.421450e+00
1     1.411697e+00
          ...     
643   3.895343e-01
644   3.265514e-01
Name: ExptRes.result0.theory prediction (fb), Length: 645, dtype: float64

In [32]:
(dataDF['xsec13TeV(fb).55'] *(3 *dataDF['BRs.y1.q,q'] + dataDF['BRs.y1.c,c'] + dataDF['BRs.y1.b,b'] ))/dataDF['ExptRes.result0.theory prediction (fb)']


0     1.000000e+00
1     1.000000e+00
          ...     
643   1.000000e+00
644   1.000000e+00
Length: 645, dtype: float64

In [33]:
mask = dataDF['ExptRes.result0.r'].isna()
rows_with_nan = dataDF[mask]
rows_with_nan
#print(rows_with_nan)


,filename,mass.55,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.55,width.5000001,width.9000002,width.9000003,width.9000004,extpar.1,extpar.2,extpar.3,extpar.4,extpar.5,extpar.6,extpar.7,extpar.8,extpar.9,extpar.10,extpar.11,extpar.12,extpar.13,extpar.14,extpar.15,extpar.16,xsec13TeV(fb).55,"BRs.y1.b,b","BRs.y1.c,c","BRs.y1.q,q","BRs.y1.t-,t+","BRs.y1.xd~,xd",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
25,run_15_mzp_1800_gvq_0.94.slha,1.800000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,8.081958e+02,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.789300e+04,1.568336e-01,1.568336e-01,1.568336e-01,1.567542e-01,5.907786e-02,1.789300e+04,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",3.861883e+03,0.000000e+00,1.789300e+04,1.403112e+04,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRV1jj_slha_broad/run_15_mzp_1800_gvq_0.94.slha,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,run_14_mzp_1800_gvq_0.63.slha,1.800000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,3.930641e+02,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.136900e+03,1.464336e-01,1.464336e-01,1.464336e-01,1.463594e-01,1.214725e-01,8.136900e+03,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.179321e+03,0.000000e+00,8.136899e+03,5.957578e+03,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRV1jj_slha_broad/run_14_mzp_1800_gvq_0.63.slha,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,run_45_mzp_2000_gvq_0.94.slha,2.000000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,8.980200e+02,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.036800e+04,1.568293e-01,1.568293e-01,1.568293e-01,1.567773e-01,5.907624e-02,1.036800e+04,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.237970e+03,0.000000e+00,1.036800e+04,8.130031e+03,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_bro

In [34]:
mask = dataDF['ExptRes.result0.r'].isna()
rows_with_nan = dataDF[mask]
rows_with_nan.to_csv('rows_with_nan_.csv', index=False)
rows_with_nan


,filename,mass.55,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.55,width.5000001,width.9000002,width.9000003,width.9000004,extpar.1,extpar.2,extpar.3,extpar.4,extpar.5,extpar.6,extpar.7,extpar.8,extpar.9,extpar.10,extpar.11,extpar.12,extpar.13,extpar.14,extpar.15,extpar.16,xsec13TeV(fb).55,"BRs.y1.b,b","BRs.y1.c,c","BRs.y1.q,q","BRs.y1.t-,t+","BRs.y1.xd~,xd",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
25,run_15_mzp_1800_gvq_0.94.slha,1.800000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,8.081958e+02,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.789300e+04,1.568336e-01,1.568336e-01,1.568336e-01,1.567542e-01,5.907786e-02,1.789300e+04,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",3.861883e+03,0.000000e+00,1.789300e+04,1.403112e+04,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRV1jj_slha_broad/run_15_mzp_1800_gvq_0.94.slha,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,run_14_mzp_1800_gvq_0.63.slha,1.800000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,3.930641e+02,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,6.339000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.136900e+03,1.464336e-01,1.464336e-01,1.464336e-01,1.463594e-01,1.214725e-01,8.136900e+03,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.179321e+03,0.000000e+00,8.136899e+03,5.957578e+03,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_broad/run_01_mzp_1800_gvq_0.13.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRV1jj_slha_broad/run_14_mzp_1800_gvq_0.63.slha,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,run_45_mzp_2000_gvq_0.94.slha,2.000000e+03,0.000000e+00,9.118760e+01,7.982436e+01,7.982436e+01,8.980200e+02,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,1.000000e+00,0.000000e+00,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,9.406900e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.036800e+04,1.568293e-01,1.568293e-01,1.568293e-01,1.567773e-01,5.907624e-02,1.036800e+04,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",2.237970e+03,0.000000e+00,1.036800e+04,8.130031e+03,0.000000e+00,5,2.000000e-01,1,TRV1jj_slha_bro